In [11]:
from pathlib import Path
import os

yr = 2024
d = 18

inp_path = os.path.join(Path(os.path.abspath("")).parents[2], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))

with open(inp_path, 'r') as file:
    inp = file.readlines()

In [12]:
import numpy as np
import heapq
from enum import Enum
from functools import cache
import copy

maxi, maxj = 70, 70 # Defines thee size of the map

def format_input(inp):
  return [(int(l.split(',')[1]), int(l.split(',')[0])) for l in inp]

In [13]:
def generate_maps(formatted_input, maxi, maxj):
  maps = [np.ones((maxi+1, maxj+1))]
  for (i, j) in formatted_input:
    cur_map = copy.deepcopy(maps[-1])
    cur_map[i][j] = np.inf
    maps.append(cur_map)
  return maps

class Directions(Enum):
    UP = 1
    DOWN = 2
    LEFT = 3
    RIGHT = 4

dir2coord = {Directions.UP: [-1,0],
            Directions.DOWN: [1,0],
            Directions.LEFT: [0,-1],
            Directions.RIGHT: [0,1]}


def turn(dir, left=False):
  d = {Directions.UP: Directions.RIGHT,
       Directions.RIGHT: Directions.DOWN,
       Directions.DOWN: Directions.LEFT,
       Directions.LEFT: Directions.UP}
  if not left:
    return d[dir]
  else:
    return {v:k for k, v in d.items()}[dir]

def arr_to_tuple(arr):
  return tuple([tuple(l) for l in arr])

def tuple_to_arr(arr):
  return np.array([(l) for l in arr])


def manhattan_distance(p1, p2):
  '''
  We use Manhattan Distance from the current location
  to the target as our A* search heuristic

  This simulates a situation where all nodes on a 
  direct path between the current and target nodes
  have an edge cost of 1
  '''
  return np.sum(np.abs(np.array(p1)-np.array(p2)))



class A_Star_Node:
  '''
   A Node which will be utilized for the A* path-finding algorithm
   If different restrictions were placed on the path then we could override the methods in this class
  '''
    
  def __init__(self, loc, direction, num_direction, g, prev, start_loc, end_loc):
    self.loc = loc
    self.direction = direction
    self.num_direction = num_direction
    self.g = g
    self.h = manhattan_distance(loc, end_loc)
    self.f = self.g + self.h
    self.prev = prev
    self.start_loc = start_loc
    self.end_loc = end_loc

  def get_descriptor(self):
    return (self.loc, self.direction, self.num_direction)


  def direction_to_next(self, direction=None):
    if direction is None:
      direction = self.direction
    dloc = dir2coord[direction]
    return tuple(np.add(dloc, list(self.loc)))

  def get_possible_next_descriptors(self):
    '''
    Return the possible next states according to
    the movement specifications

    We break the unique A* state nodes up by:
    (location, direction, number moved in the direction)

    i.e. Arriving at node (2,2) after going right 2 times is a
    DIFFERENT state than arriving at node (2,2) afterg going right 3 times

    We structure our search of the state space accordingly
    '''
      
    # If we are at the end location, then return no new locations
    if self.loc == self.end_loc:
      return []

    possible_nexts = [(self.direction_to_next(d), d, 1)
                      for d in list(Directions)]


    # If one of the nexts is the end then convert it to the end
    return [n if n[0]!=self.end_loc else (n[0], None, None) for n in possible_nexts]

  def __eq__(self, other):
    if isinstance(other, tuple):
      return self.get_descriptor() == other
    if isinstance(other, A_Star_Node):
      return self.get_descriptor() == other.get_descriptor()
    return None

  def __hash__(self):
    return hash(self.get_descriptor())


def a_star(start, end, arr, A_Star_Node=A_Star_Node, start_dir=Directions.RIGHT):
  '''
  A* Implementation with a priority queue for the queue nodes

  Also implemented hash tables for fast lookups in queue, and searched lists
  '''

  class Heap_Node_Wrapper:
    '''
    Wraps around a heap node so the priority
    queue can order them by their F and H values
    '''
    def __init__(self, node):
      self.node = node

    def __lt__(self, other):
      n = self.node
      other = other.node
      return (n.f, n.h) < (other.f, other.h)

    def __eq__(self, other):
      n = self.node
      other = other.node
      return (n.f, n.h) == (other.f, other.h)

  def in_arr(loc):
    h, w = arr.shape
    return ((0 <= loc[0] and loc[0] < h)
            and (0 <= loc[1] and loc[1] < w))

  s_node = A_Star_Node(start, start_dir, None, arr[start], None, start, end)
  q = [Heap_Node_Wrapper(s_node)]
  qhash = {s_node:s_node} # Mirrors q, just so we can do O(1) existence checks
  searched = {}

  cnt = 0
  while len(q) != 0:

    cur = heapq.heappop(q).node
    del qhash[cur]
    searched[cur]=True

    for n in [x for x in cur.get_possible_next_descriptors() if x not in searched and in_arr(x[0])]:
        
      # n is just a descriptor so if we see a node in the queue matching that
      # descriptor then we can get a reference to it
      n_node = None
      if n in qhash:
        n_node = qhash[n]

      ng = cur.g + arr[n[0]]

      if n_node is None or ng < n_node.g:
        if n_node is None:
          # If we don't have this node in the queue create it
          n_node = A_Star_Node(n[0], n[1], n[2], ng, cur, start, end)
          heapq.heappush(q, Heap_Node_Wrapper(n_node))
          qhash[n_node] = n_node
        else:
          n_node.g = ng
          n_node.prev = cur
    cnt+=1

  # Once we make it to the end node trace back through
  # the path to the beginning
  endns = [s for s in searched if s.loc==end]
  assert(len(endns)==1)
  endn = endns[0]
  path = [endn.get_descriptor()]
  at_beginning = False
  while not at_beginning:
    if endn.prev is not None:
      path.append(endn.prev.get_descriptor())
    endn = endn.prev
    if endn.prev is None:
      at_beginning = True

  while(len(path)>=2 and path[0][0]==end and path[1][0]==end):
    path = path[1:]

  return list(reversed(path))


@cache
def get_cost(maze, path):
  '''
  Given a path return, the cost
  of that path
  '''
  if isinstance(path, tuple):
    path = list(path)
  c = 0

  path_list = path
  for i, p in enumerate(path_list):
    c += maze[p[0]]
  return c

def print_path(maze, path):
  cost = get_cost(maze, tuple(path))
  if isinstance(maze, tuple):
    maze = tuple_to_arr(maze)
  maze = maze.astype(str)
  maze = np.vectorize(lambda x: '#' if x == 'inf' else '.' if x in ['1.0', '1'] else '#')(maze)
  d2c = {Directions.UP: '^',
         Directions.LEFT: '<',
         Directions.RIGHT: '>',
         Directions.DOWN: 'v'}
  for p in path:
    maze[p[0]] = d2c.get(p[1], '.')
  s = '\n'.join([''.join(l) for l in maze])
  print(s)
  print(f"(Path took {len(path)-1} Steps with a Total Cost of {int(cost)})")


@cache
def get_cost(maze, path):
  '''
  Given a path return, the cost
  of that path
  '''
  if isinstance(path, tuple):
    path = list(path)
  if isinstance(maze, tuple):
    maze = tuple_to_arr(maze)
  c = 0

  path_list = path
  for i, p in enumerate(path_list):
    if i != 0:
      c += maze[p[0]]
  return c



def guess_num(past_guesses=[], min_guess=0, max_guess=100000000, is_lower_condition=lambda guess: guess<123456, is_higher_condition=lambda guess: guess>123456, condition=lambda guess: guess==123456):
  if len(past_guesses) == 0:
    cur_guess = min_guess
  if len(past_guesses) == 1:
    cur_guess = max_guess
  else:
    lowers = [g[0] for g in past_guesses if g[1]=='l']
    highers = [g[0] for g in past_guesses if g[1]=='h']
    highest_lower = max(lowers) if len(lowers) != 0 else min_guess
    lowest_higher = min(highers) if len(highers) != 0 else max_guess
    cur_guess = highest_lower + (lowest_higher-highest_lower)//2

  if condition(cur_guess):
    return cur_guess
  elif is_lower_condition(cur_guess):
    return guess_num(past_guesses+[(cur_guess, 'l')], min_guess, max_guess, is_lower_condition, is_higher_condition, condition)
  elif is_higher_condition(cur_guess):
    return guess_num(past_guesses+[(cur_guess, 'h')], min_guess, max_guess, is_lower_condition, is_higher_condition, condition)
  else:
    raise Exception("Oops")
  
def determine_first_blocking_byte(formatted_input):
  maps = generate_maps(formatted_input, maxi, maxj)
  start = (0,0)
  end = (maxi, maxj)
  
  def is_lower_condition(i):
    path = a_star(start, end, maps[i], A_Star_Node=A_Star_Node)
    cost = get_cost(arr_to_tuple(maps[i]), tuple(path))
    if np.isinf(cost):
      return False
    if i+1 >= len(maps):
      return False
    pathn = a_star(start, end, maps[i+1], A_Star_Node=A_Star_Node)
    costn = get_cost(arr_to_tuple(maps[i+1]), tuple(pathn))
    if np.isinf(costn):
      return False
    return True
  
  def is_higher_condition(i):
    path = a_star(start, end, maps[i], A_Star_Node=A_Star_Node)
    cost = get_cost(arr_to_tuple(maps[i]), tuple(path))
    if not np.isinf(cost):
      return False
    elif i+1 >= len(maps):
      return True
    pathn = a_star(start, end, maps[i+1], A_Star_Node=A_Star_Node)
    costn = get_cost(arr_to_tuple(maps[i+1]), tuple(pathn))
    if not np.isinf(costn):
      return False
    return True
  
  def condition(i):
    path = a_star(start, end, maps[i], A_Star_Node=A_Star_Node)
    cost = get_cost(arr_to_tuple(maps[i]), tuple(path))
    if np.isinf(cost):
      return False
    elif i+1 >= len(maps):
      return True
    pathn = a_star(start, end, maps[i+1], A_Star_Node=A_Star_Node)
    costn = get_cost(arr_to_tuple(maps[i+1]), tuple(pathn))
    if not np.isinf(costn):
      return False
    return True
  
  out = guess_num(min_guess=0, max_guess=len(maps)-1, is_lower_condition=is_lower_condition, is_higher_condition=is_higher_condition, condition=condition)
  return ','.join(map(str,reversed(formatted_input[out])))

In [14]:
import time

t = time.time()

formatted_input = format_input(inp)
start = (0,0)
end = (maxi, maxj)
maps = generate_maps(formatted_input, maxi, maxj)

print_path(arr_to_tuple(maps[1024]), a_star(start, end, maps[1024], A_Star_Node=A_Star_Node))
print(determine_first_blocking_byte(formatted_input))

print('\nRUNTIME: ', time.time()-t)

>>>>>>>#...#...........#.........#.#...................................
######v#.#.#.#######.#.###.#####.#.#...#...............................
<<<<<<v#.#...#.....#.#.....#.......#...#...............................
v###.###.####..#...#.###.#####.######..#...............................
v#...#...#.....#.#.#.#...#...#...#.....#...............................
v#.###.#########.###.#####.#.###.#.#####.##............................
v#...#.....#^>>#...#.......#...#.#...#...#.#...........................
v#########.#^#v#.#.###########.#####.#.#.....#.........................
v..........#^#v#.#...#...#.....#...#.#.#.#.............................
v###########^#v.#....#.#.#.#####.#.#.#.#...............................
v>>>>>>#^>>>>#v#...#.#.#...#.....#...#.#...............................
######v#^#####v#.#.##..#####.######.##.#...............................
.....#v>>#...#v#.....#.......#.....#...................................
.#.#######..##v###.#.#########..##.####.........................